In [9]:
import pandas as pd
import numpy as np
import os
import glob
import json

VELOCITY_THRESHOLD = 100  # Degrees per second

def load_csv_files(path):
    """Load all CSV files in a directory."""
    all_files = glob.glob(os.path.join(path, "*.csv"))
    data_frames = {os.path.basename(f): pd.read_csv(f) for f in all_files}
    return data_frames


def calculate_velocity(data):
    data['velocity'] = np.sqrt((data['x'].diff()**2) + (data['y'].diff()**2)) / data['timestamp'].diff()
    return data

def ivt_algorithm(data):
    data = calculate_velocity(data)
    data['fixation'] = data['velocity'] <= VELOCITY_THRESHOLD
    
    grouped_fixations = []
    grouped_saccades = []
    fixation_start = None

    for i, row in data.iterrows():
        if row['fixation']:
            if fixation_start is None:
                fixation_start = i  # Start of a new fixation
        else:
            grouped_saccades.append([data['x'].iloc[i].item(), data['y'].iloc[i].item()])
            if fixation_start is not None:
                # End of the fixation
                fixation_group = data.iloc[fixation_start:i]
                grouped_fixations.append([
                    fixation_group['x'].iloc[0].item(),  # x_start
                    fixation_group['y'].iloc[0].item(),  # y_start
                    fixation_group['x'].iloc[-1].item(),  # x_end
                    fixation_group['y'].iloc[-1].item(),  # y_end
                ])
                fixation_start = None

    # Handle the case where a fixation ends at the last point
    if fixation_start is not None:
        fixation_group = data.iloc[fixation_start:]
        grouped_fixations.append([
            fixation_group['x'].iloc[0].item(),  # x_start
            fixation_group['y'].iloc[0].item(),  # y_start
            fixation_group['x'].iloc[-1].item(),  # x_end
            fixation_group['y'].iloc[-1].item(),  # y_end
        ])
    
    return grouped_fixations, grouped_saccades, data

def process_all_files(input_path, output_path):
    """Process all CSV files in the directory."""
    all_data = load_csv_files(input_path)
    for file_name, df in all_data.items():
        df = df.dropna()  # Clean missing data
        fixations, saccades, df = ivt_algorithm(df)
        df.to_csv(os.path.join(output_path, file_name), index=False)
        file_fix = file_name.replace(".csv", "_fixations.json")
        file_fix_path = os.path.join(output_path, file_fix)
        with open(file_fix_path, 'w') as f:
            json.dump(fixations, f)
        file_sac = file_name.replace(".csv", "_saccades.json")
        file_sac_path = os.path.join(output_path, file_sac)
        with open(file_sac_path, 'w') as f:
            json.dump(saccades, f)
        print(f"Processed and saved: {file_name}")


name_set = {'participant1': [1,3,4,5,6,7,8,9,10], 'participant2': [1,2,4,5,6,7,8,9,10], 'participant3': [1,2,3,4,5,6,7,9,10], 
            'participant4': [1,2,3,4,6,7,8,9,10], 'participant5': [1,2,3,5,6,7,8,9,10], 'participant6': [1,2,3,4,5,6,8,9,10], 
            'participant7': [1,2,3,4,5,7,8,9,10], 'participant8': [2,3,4,5,6,7,8,9,10]}
names = ['participant1', 'participant2', 'participant3', 'participant4', 'participant5', 'participant6', 'participant7', 'participant8']
for name in names:
    for s in name_set[name]:
        input_path = r"C:/Users/lack_/Desktop/et-mt-evaluation-project/process_gaze/mappings/{}/set{}/corrected/traces".format(name,s)
        output_path = r"C:/Users/lack_/Desktop/et-mt-evaluation-project/process_gaze/mappings/{}/set{}/corrected/traces/output".format(name,s)
        os.makedirs(output_path, exist_ok=True)
        process_all_files(input_path, output_path)

Processed and saved: coords_with_time1.csv
Processed and saved: coords_with_time10.csv
Processed and saved: coords_with_time2.csv
Processed and saved: coords_with_time3.csv
Processed and saved: coords_with_time4.csv
Processed and saved: coords_with_time5.csv
Processed and saved: coords_with_time6.csv
Processed and saved: coords_with_time7.csv
Processed and saved: coords_with_time8.csv
Processed and saved: coords_with_time9.csv
Processed and saved: coords_with_time1.csv
Processed and saved: coords_with_time10.csv
Processed and saved: coords_with_time2.csv
Processed and saved: coords_with_time3.csv
Processed and saved: coords_with_time4.csv
Processed and saved: coords_with_time5.csv
Processed and saved: coords_with_time6.csv
Processed and saved: coords_with_time7.csv
Processed and saved: coords_with_time8.csv
Processed and saved: coords_with_time9.csv
Processed and saved: coords_with_time1.csv
Processed and saved: coords_with_time10.csv
Processed and saved: coords_with_time2.csv
Processe